# postprocess results

In [ ]:
data = []
for r in responses:
    if "data" in r and isinstance(r["data"],pd.core.frame.DataFrame):
        if "docid" not in r["data"].columns:
            r["data"].columns = ["first name", "last name", "date of birth"]
            r["data"]["docid"] = r["image"]
        data.append(r["data"])
data = pd.concat(data,axis=0, ignore_index=True)
data.columns = ["strGname", "strLname", "DateOfBirth","docid"]

In [ ]:
data["docid"] = data["docid"].apply(lambda name: name.replace(".jpg","").split("/")[-1])

In [ ]:
with open(f"transcriptions/transcriptions_data.pkl", "wb") as file:
    pickle.dump(data, file)

In [ ]:
with open(f"transcriptions/transcriptions_data.pkl", "rb") as file:
    data = pickle.load(file)

In [ ]:
# dump results
#with open("transcriptions.pkl", "wb") as file:
#    pickle.dump(responses, file)

## Match

In [ ]:
from aroa_etl.attribute_processing.string_utils import preprocess_name, preprocess_last_name
from collections import defaultdict
from tqdm import tqdm

In [ ]:
transcription = data

In [ ]:
transcription["strLName_processed"] = transcription["strLname"].fillna("").apply(preprocess_last_name)
transcription["strGName_processed"] = transcription["strGname"].fillna("").apply(preprocess_name)
transcription["DateOfBirth"] = transcription["DateOfBirth"].str.replace(r'^[^\d]*$', '00000000', regex=True)
transcription["strDoB_processed"] = transcription["DateOfBirth"].fillna('00000000')
transcription["strDoB_processed"] = transcription["strDoB_processed"].apply(lambda d: f"{d[6:8]}.{d[4:6]}.{d[0:4]}" if re.match(r"^\d\d\d\d\d\d\d\d$", d) else d)
transcription = transcription.drop_duplicates()

In [ ]:
transcription.to_pickle("transcriptions/transcriptions_processed.pkl")

In [ ]:
table_name = ""
known_names = pd.read_excel(table_name)

In [ ]:
known_names["strDoB_processed"] = known_names["Birth Date Txt"].fillna("00000000").astype(str).fillna("00000000")
known_names["strLName_processed"] = known_names["Family Name"].fillna("").astype(str).apply(preprocess_last_name)
known_names["strGName_processed"] = known_names["First Name"].fillna("").astype(str).apply(preprocess_name)

In [ ]:
from aroa_etl.person_matching.matching import person_matching

In [ ]:
import importlib
import aroa_etl
importlib.reload(aroa_etl.person_matching.similarity_measures)

In [ ]:
known_names.columns

In [ ]:
matchings_name_only_df = person_matching(transcription, known_names, 
                     src_gname_col="strGName_processed", target_gname_col="strGName_processed",
                     src_lname_col="strLName_processed", target_lname_col="strLName_processed",
                     src_date_col="strDoB_processed", target_date_col="strDoB_processed",
                    trg_pre_clustering_on_n_chars=1, trg_pre_clustering_group_n_len_units=4,
                    top_n_matches = 1, min_match_score=0.0, name_only=True)
matchings_name_only_df.score.hist()

In [ ]:
(matchings_name_only_df.score >= 80).sum() / matchings_name_only_df.shape[0]

In [ ]:
name_matched = matchings_name_only_df.set_index("srcID").score >= 80.0

In [ ]:
matchings = person_matching(transcription, known_names,  
                    src_gname_col="strGName_processed", target_gname_col="strGName_processed",
                    src_lname_col="strLName_processed", target_lname_col="strLName_processed",
                    src_date_col="strDoB_processed", target_date_col="strDoB_processed",
                    trg_pre_clustering_on_n_chars=1, trg_pre_clustering_group_n_len_units=4,
                    top_n_matches = 1, min_match_score=0.0)

In [ ]:
matchings_df = matchings #pd.DataFrame(matchings,columns=["srcID","score","trgID"])

In [ ]:
(matchings_df.score >= 80).sum() / matchings_df.shape[0]

In [ ]:
matchings_df.to_pickle("transcriptions/matchings_df.pkl")

In [ ]:
matchings_df = pd.read_pickle("transcriptions/matchings_df.pkl")
transcription = pd.read_pickle("transcriptions/transcriptions_processed.pkl")

In [ ]:
matchings_df.score.hist()

In [ ]:
# matched
is_match = transcription_matched.score>=80
transcription_matched["is_matched"] = is_match
transcription_matched['strGName_matched'] = transcription_matched['First Name_match']
transcription_matched['strLName_matched'] = transcription_matched['Family Name_match']
transcription_matched['strDoB_matched'] = transcription_matched['Birth Date Txt_match']
transcription_matched.loc[~is_match,'First Name_match'] = transcription_matched.loc[~is_match,'strGname']
transcription_matched.loc[~is_match,'Family Name_match'] = transcription_matched.loc[~is_match,'strLname']
transcription_matched.loc[~is_match,'Birth Date Txt_match'] = transcription_matched.loc[~is_match,'DateOfBirth']
transcription_matched.drop(
    ['strGName_processed','strGName_processed_match',
     'strLName_processed','strLName_processed_match',
     'strDoB_processed','strDoB_processed_match'],inplace=True,axis="columns")
transcription_matched.columns = [re.sub(r"_match$","",col) for col in transcription_matched.columns]

In [ ]:
print(f"merge matching indices")
transcription_matched = pd.merge(transcription, matchings_df, left_index=True, right_on='srcID')
transcription_matched.to_pickle("transcriptions/transcriptions_matched.pkl")
#transcription_matched = 
transcription_matched["is_ambiguous"] = False 
is_qa = transcription_matched["score"] < 80
transcription_matched.loc[is_qa, "is_ambiguous"] = True

no_match = transcription_matched["score"] < 60
transcription_matched.loc[no_match, "trgID"] = None

print(f"merge matching data")
known_names.columns = [f"{c}_match" for c in known_names.columns]
transcription_matched.trgID = transcription_matched.trgID.apply(lambda idx: int(idx) if pd.notna(idx) else idx)
transcription_matched = pd.merge(transcription_matched,known_names, how="left", left_on="trgID",right_index=True,)
transcription_matched.drop(["srcID","trgID"],inplace=True, axis="columns")

transcription_matched = transcription_matched.fillna("")
# attach name matched column
transcription_matched = pd.merge(transcription_matched,name_matched.to_frame().rename({"score": "NameMatch"},axis=1), how="left",left_index=True,right_index=True,)
print(f"save results")
transcription_matched.to_pickle("transcriptions/transcription_matched.pkl")
transcription_matched.to_csv("transcriptions/transcription_matched.csv")
#transcription_matched.to_excel("transcription_matched.xls")
qa_ratio = transcription_matched["is_ambiguous"].mean()

print(f"Not Matched Name+Date {qa_ratio}; Not Matched Name Only {transcription_matched.NameMatch.mean()}")

# postprocess

In [ ]:
from itables import show
import pandas as pd
import re
transcription_matched = pd.read_pickle("transcriptions/transcription_matched.pkl").fillna("")

In [ ]:
from aroa_etl.attribute_processing.column_processing import normalize_last_name, normalize_first_name, normalise_prisoner_number, normalise_date

In [ ]:
transcription_matched.rename(
    {
        "First Name":"strGName",
        "Family Name":"strLName",
        'Birth Date Txt': 'strDoB',
        'Birth City': 'strPoB',
        'Inmate: Prisoner ID':'prisoner_number',
        'strGname': 'strGname_raw',
        'strLname': 'strLname_raw',
        'DateOfBirth': 'strDoB_raw'
    }, inplace=True, axis="columns"
)

In [ ]:
show(transcription_matched)

In [ ]:
#transcription_matched = transcription_matched.astype(str)
transcription_matched["strDoB"] = transcription_matched["strDoB"].apply(repair_date)
transcription_matched["strGName"] = transcription_matched["strGName"].apply(remove_numbers_from_string)
transcription_matched["strLName"] = transcription_matched["strLName"].apply(remove_numbers_from_string)
date_df = normalise_date(transcription_matched,"strDoB","strDoB","KI Indizierung",[1855,1945])
for c in date_df.columns:
    transcription_matched[c] = date_df[c]
fname_df = normalize_first_name(transcription_matched, "strGName", "KI Indizierung")
for c in fname_df.columns:
    transcription_matched[c] = fname_df[c]
lname_df = normalize_last_name(transcription_matched, "strLName", "KI Indizierung")
for c in lname_df.columns:
    transcription_matched[c] = lname_df[c]
transcription_matched.loc[transcription_matched['last_name_birth_name_extracted'].isna(),'last_name_birth_name_extracted'] =  transcription_matched.loc[transcription_matched['last_name_birth_name_extracted'].isna(),'Maiden Name']
transcription_matched.drop(['Maiden Name'],axis="columns",inplace=True)
prisoner_df = normalise_prisoner_number(transcription_matched,"prisoner_number", "KI Indizierung")
for c in prisoner_df.columns:
    transcription_matched[c] = prisoner_df[c]
transcription_matched = transcription_matched.fillna("").astype(str)
transcription_matched["prisoner_number_qa"] = "False"
source_cols = [col for col in transcription_matched.columns if "source" in col]
for col in source_cols:
    transcription_matched.loc[is_match,col] = "Datenaquise Matching"

In [ ]:
qa_cols = [c for c in transcription_matched.columns if "_qa" in c]
transcription_matched["is_ambiguous"] = False
ambiguous_cols = [[] for i in range(transcription_matched.shape[0])]
for qa_col in qa_cols:
    transcription_matched["is_ambiguous"] = transcription_matched["is_ambiguous"] | (transcription_matched[qa_col] == True)
    col_name = qa_col.replace("_qa","")
    ambiguous_cols = map(lambda lists: [*lists[0],*lists[1]],
                            zip(ambiguous_cols, 
                                list(transcription_matched[qa_col].apply(lambda b: [col_name] if b=="True" else [] ))))
ambiguous_cols = [", ".join(col_list) for col_list in ambiguous_cols]
transcription_matched["ambiguous_columns"] = ambiguous_cols

In [ ]:
show(transcription_matched)

In [ ]:
transcription_matched.to_excel("transcriptions/indizierung.xlsx")
transcription_matched.to_csv("transcriptions/indizierung.csv",sep='|')